# 3 Create Routed Paths Duarouter

The goal of the following notebook is to compute the Routed Paths from a Mobility Demand using Duarouter as the routing algorithm.
___
We translate the $N$ trips in the Mobility Demand $D$ into $N$ paths obtaining a multiset $\overline{D}$ of routed paths within the urban environment. 
Each path $P_v(e_o,e_d, R) {=} (e_o, \dots, e_d)$ of a vehicle $v$
is a sequence of edges on the road network connecting $e_o$ and $e_d$, obtained by some routing algorithm $R$. 
When a vehicle's path is generated by a routing algorithm $R$, we say that the vehicle is $R$-routed.
In $\overline{D} = {\{P_1, \dots, P_N\}}$, the routed paths are generated independently by (different) routing algorithms.
___

In [ ]:
from utils import *

#import warnings
#warnings.filterwarnings("ignore")

#### File paths

In [ ]:
# mobility demand path
mobility_demand_filename = "../data/mobility_demand/Milano_big/Milano_big_MD_30k.rou.xml"

# road network path
road_network_path = "../data/road_net/Milano_big/Milano_big_road_network.net.xml"

# output filename prefix
output_demand_filename_prefix = "../data/simulations/Milano_big/baseline/routed_paths/Milano_big_TD_30k" 

In [ ]:
# number of traffic demand to generate
experiments = 10

In [ ]:
# output filename 
output_demand_filename_experiments = []
for e in range(experiments):
    output_demand_filename_experiments.append(output_demand_filename_prefix+'_'+str(e)+'.rou.xml')

In [ ]:
len(output_demand_filename_experiments)

#### Set duarouter's parameters

In [ ]:
weight = 7.5
rm_loops = "false"

seed_duarouter_list = []
seed_duarouter = None

if seed_duarouter is None:
    for e in range(experiments):
        seed_duarouter = np.random.randint(0, 9999999)
        seed_duarouter_list.append(seed_duarouter)
else:
    seed_duarouter_list = [seed_duarouter]*experiments

options_duarouter = "--weights.random-factor "+str(weight)+" --max-alternatives 10 --remove-loops "+rm_loops+" "\
    "--weights.interpolate true --weights.minor-penalty 0 "\
    " --routing-threads 8"

In [ ]:
len(seed_duarouter_list)

#### Apply duarouter

Execution time with experiments = 10 and n_vehicles=30k $\approx 10 min$

In [ ]:
for e in range(experiments):

    # prepare the command string for duarouter
    
    command_str = "duarouter --route-files "+mobility_demand_filename+" "+\
            " --net-file "+road_network_path+" "+options_duarouter+\
        " --random false --seed "+str(seed_duarouter_list[e])+\
        " --output-file "+output_demand_filename_experiments[e]

    # call duarouter process
    call_duarouter_command(command_str)

    # remove .alt file
    os.remove(output_demand_filename_experiments[e].split(".rou")[0]+".rou.alt"+output_demand_filename_experiments[e].split(".rou")[1])

    print("Created the routed demand [duarouter]: "+output_demand_filename_experiments[e])